In [1]:
# Libraries
import itertools
from importlib import resources
import pandas as pd
import numpy as np
from tqdm import tqdm

from drdt.helper_functions import DecisionRuleCreatorFromDecisionTable, Reduction, R_SR, R_AD, SAlphaStep, SPlus, SMax, NCover, NGreedy
from drdt.algorithms import DynamicProgrammingAlgorithms, A_C_N, A_C_G


# Loading Data

In [2]:
with resources.path('datasets.DecisionRuleSystems', 'DRS_congressional_voting_records') as dataset_path:
    S = pd.read_csv(dataset_path).sample(n=15, random_state=42).applymap(lambda x: str(x) if pd.notnull(x) else x)
S = S.dropna(axis=1, how='all') # Drop the columns with all None
S

,handicapped-infants,water-project-cost-sharing,adoption-of-the-budget-resolution,physician-fee-freeze,anti-satellite-test-ban,aid-to-nicaraguan-contras,mx-missile,synfuels-corporation-cutback,crime,export-administration-act-south-africa,class
30,NaN,n,n,y,NaN,NaN,NaN,NaN,NaN,NaN,republican
126,NaN,NaN,NaN,n,NaN,NaN,NaN,NaN,n,NaN,democrat
199,NaN,NaN,NaN,n,NaN,NaN,NaN,NaN,NaN,NaN,democrat
142,NaN,NaN,NaN,n,NaN,n,NaN,NaN,NaN,NaN,democrat
254,NaN,NaN,NaN,n,NaN,NaN,NaN,NaN,NaN,NaN,democrat
237,NaN,NaN,NaN,n,NaN,NaN,NaN,NaN,NaN,NaN,democrat
97,NaN,NaN,n,y,NaN,NaN,NaN,n,NaN,NaN,republican
206,NaN,n,NaN,y,NaN,NaN,NaN,NaN,NaN,NaN,republican
258,NaN,NaN,NaN,n,NaN,NaN,NaN,NaN,NaN,NaN,democrat
144,NaN,NaN,NaN,n,NaN,NaN,n,NaN,NaN,NaN,democrat


# Dataset Analyses

In [3]:
print(f"Number of features  = {len(S.columns)-1}")

Number of features  = 10


In [4]:
print(f"Does # of features = d  = {not S.dropna().empty}")

Does # of features = d  = False


In [5]:
# Count non-NaN values for each row
non_nan_counts = S.count(axis=1)

# Find the index
max_non_nan_row_index = non_nan_counts.idxmax()

# Retrieve the row
max_non_nan_row = S.loc[max_non_nan_row_index]

# Number of non-NaN values in the row
max_non_nan_count = non_nan_counts[max_non_nan_row_index]

print(f"d of  = {max_non_nan_count - 1}") # remove 1 because of last class column

d of  = 3


In [6]:
P = S
P_plus = SPlus(P)
B = NCover(P_plus)
print(f"Length of Node Cover for AR = {len(B)}")

Length of Node Cover for AR = 6


In [7]:
P = R_SR(S)
P_plus = SPlus(P)
B = NCover(P_plus)
print(f"Length of Node Cover for SR = {len(B)}")

Length of Node Cover for SR = 3


In [8]:
P = R_AD(S)
P_plus = SPlus(P)
B = NCover(P_plus)
print(f"Length of Node Cover for AD = {len(B)}")

Length of Node Cover for AD = 3


In [9]:
P = S
P_plus = SPlus(P)
B = NGreedy(P_plus)
print(f"Length of Node Cover greedy for AR = {len(B)}")

Length of Node Cover greedy for AR = 2


In [10]:
P = R_SR(S)
P_plus = SPlus(P)
B = NGreedy(P_plus)
print(f"Length of Node Cover greedy for SR = {len(B)}")

Length of Node Cover greedy for SR = 2


In [11]:
P = R_AD(S)
P_plus = SPlus(P)
B = NGreedy(P_plus)
print(f"Length of Node Cover greedy for AD = {len(B)}")

Length of Node Cover greedy for AD = 2


# Dynamic Programming

In [12]:
alg = DynamicProgrammingAlgorithms(C="AR")
depth = alg.A_DP(S)
print(f"DP AR = {depth}")

DP AR = 7


In [13]:
alg = DynamicProgrammingAlgorithms(C="EAR")
depth = alg.A_DP(S)
print(f"DP EAR = {depth}")

DP EAR = 7


In [14]:
alg = DynamicProgrammingAlgorithms(C="SR")
depth = alg.A_DP(S)
print(f"DP SR = {depth}")

DP SR = 2


In [15]:
alg = DynamicProgrammingAlgorithms(C="ESR")
depth = alg.A_DP(S)
print(f"DP ESR = {depth}")

DP ESR = 7


In [16]:
alg = DynamicProgrammingAlgorithms(C="AD")
depth = alg.A_DP(S)
print(f"DP AD = {depth}")

DP AD = 4


In [17]:
alg = DynamicProgrammingAlgorithms(C="EAD")
depth = alg.A_DP(S)
print(f"DP EAD = {depth}")

DP EAD = 7


# Combinations

In [18]:
column_values = []

for column in S.columns[:-1]: #ignoring class column
    column_values.append(list(S[column].dropna().unique()))
    
# All possible combinations    
combinations = list(itertools.product(*column_values))


In [19]:
column_values_extension = []

for column in S.columns[:-1]: #ignoring class column
    column_values_extension.append(list(S[column].dropna().unique()) + ['*'])
    
# All possible combinations with extension    
extended_combinations = list(itertools.product(*column_values_extension))


# Node Cover

In [20]:
depths = []

for comb in tqdm(combinations):
    #creating delta
    delta = pd.DataFrame([list(comb)], columns=list(S.columns[:-1]))
    
    delta = delta.loc[0]
    
    alg = A_C_N(C="AR", N="cover")
    
    depth, _ = alg.solve(S, delta=delta)
    
    depths.append(depth)
    
print("NC AR")    
print("(Max Depth, Min Depth, Average Depth) =", (max(depths), min(depths), sum(depths)/len(depths)))

100%|█████████████████████████████████████████████| 2/2 [00:00<00:00, 81.70it/s]

NC AR
(Max Depth, Min Depth, Average Depth) = (9, 7, 8.0)


In [21]:
depths = []

for comb in tqdm(extended_combinations):
    #creating delta
    delta = pd.DataFrame([list(comb)], columns=list(S.columns[:-1]))
    
    delta = delta.loc[0]
    
    alg = A_C_N(C="EAR", N="cover")
    
    depth, _ = alg.solve(S, delta=delta)
    
    depths.append(depth)
    
print("NC EAR")    
print("(Max Depth, Min Depth, Average Depth) =", (max(depths), min(depths), sum(depths)/len(depths)))

100%|██████████████████████████████████████| 1536/1536 [00:11<00:00, 129.36it/s]

NC EAR
(Max Depth, Min Depth, Average Depth) = (9, 6, 7.166666666666667)


In [22]:
depths = []

for comb in tqdm(combinations):
    #creating delta
    delta = pd.DataFrame([list(comb)], columns=list(S.columns[:-1]))
    
    delta = delta.loc[0]
    
    alg = A_C_N(C="SR", N="cover")
    
    depth, _ = alg.solve(S, delta=delta)
    
    depths.append(depth)
    
print("NC SR")    
print("(Max Depth, Min Depth, Average Depth) =", (max(depths), min(depths), sum(depths)/len(depths)))

100%|████████████████████████████████████████████| 2/2 [00:00<00:00, 123.18it/s]

NC SR
(Max Depth, Min Depth, Average Depth) = (3, 3, 3.0)


In [23]:
depths = []

for comb in tqdm(extended_combinations):
    #creating delta
    delta = pd.DataFrame([list(comb)], columns=list(S.columns[:-1]))
    
    delta = delta.loc[0]
    
    alg = A_C_N(C="ESR", N="cover")
    
    depth, _ = alg.solve(S, delta=delta)
    
    depths.append(depth)
    
print("NC ESR")    
print("(Max Depth, Min Depth, Average Depth) =", (max(depths), min(depths), sum(depths)/len(depths)))

100%|██████████████████████████████████████| 1536/1536 [00:14<00:00, 106.76it/s]

NC ESR
(Max Depth, Min Depth, Average Depth) = (7, 3, 4.041666666666667)


In [24]:
depths = []

for comb in tqdm(combinations):
    #creating delta
    delta = pd.DataFrame([list(comb)], columns=list(S.columns[:-1]))
    
    delta = delta.loc[0]
    
    alg = A_C_N(C="AD", N="cover")
    
    depth, _ = alg.solve(S, delta=delta)
    
    depths.append(depth)
    
print("NC AD")    
print("(Max Depth, Min Depth, Average Depth) =", (max(depths), min(depths), sum(depths)/len(depths)))

100%|████████████████████████████████████████████| 2/2 [00:00<00:00, 120.23it/s]

NC AD
(Max Depth, Min Depth, Average Depth) = (5, 3, 4.0)


In [25]:
depths = []

for comb in tqdm(extended_combinations):
    #creating delta
    delta = pd.DataFrame([list(comb)], columns=list(S.columns[:-1]))
    
    delta = delta.loc[0]
    
    alg = A_C_N(C="EAD", N="cover")
    
    depth, _ = alg.solve(S, delta=delta)
    
    depths.append(depth)
    
print("NC EAD")    
print("(Max Depth, Min Depth, Average Depth) =", (max(depths), min(depths), sum(depths)/len(depths)))

100%|██████████████████████████████████████| 1536/1536 [00:12<00:00, 122.37it/s]

NC EAD
(Max Depth, Min Depth, Average Depth) = (7, 3, 4.375)


# Node Cover Greedy

In [26]:
depths = []

for comb in tqdm(combinations):
    #creating delta
    delta = pd.DataFrame([list(comb)], columns=list(S.columns[:-1]))
    
    delta = delta.loc[0]
    
    alg = A_C_N(C="AR", N="greedy")
    
    depth, _ = alg.solve(S, delta=delta)
    
    depths.append(depth)
    
print("NCgreedy AR")    
print("(Max Depth, Min Depth, Average Depth) =", (max(depths), min(depths), sum(depths)/len(depths)))

100%|█████████████████████████████████████████████| 2/2 [00:00<00:00, 52.67it/s]

NCgreedy AR
(Max Depth, Min Depth, Average Depth) = (7, 7, 7.0)


In [27]:
depths = []

for comb in tqdm(extended_combinations):
    #creating delta
    delta = pd.DataFrame([list(comb)], columns=list(S.columns[:-1]))
    
    delta = delta.loc[0]
    
    alg = A_C_N(C="EAR", N="greedy")
    
    depth, _ = alg.solve(S, delta=delta)
    
    depths.append(depth)
    
print("NCgreedy EAR")    
print("(Max Depth, Min Depth, Average Depth) =", (max(depths), min(depths), sum(depths)/len(depths)))

100%|███████████████████████████████████████| 1536/1536 [00:20<00:00, 74.60it/s]

NCgreedy EAR
(Max Depth, Min Depth, Average Depth) = (7, 2, 4.666666666666667)


In [28]:
depths = []

for comb in tqdm(combinations):
    #creating delta
    delta = pd.DataFrame([list(comb)], columns=list(S.columns[:-1]))
    
    delta = delta.loc[0]
    
    alg = A_C_N(C="SR", N="greedy")
    
    depth, _ = alg.solve(S, delta=delta)
    
    depths.append(depth)
    
print("NCgreedy SR")    
print("(Max Depth, Min Depth, Average Depth) =", (max(depths), min(depths), sum(depths)/len(depths)))

100%|█████████████████████████████████████████████| 2/2 [00:00<00:00, 46.43it/s]

NCgreedy SR
(Max Depth, Min Depth, Average Depth) = (4, 4, 4.0)


In [29]:
depths = []

for comb in tqdm(extended_combinations):
    #creating delta
    delta = pd.DataFrame([list(comb)], columns=list(S.columns[:-1]))
    
    delta = delta.loc[0]
    
    alg = A_C_N(C="ESR", N="greedy")
    
    depth, _ = alg.solve(S, delta=delta)
    
    depths.append(depth)
    
print("NCgreedy ESR")    
print("(Max Depth, Min Depth, Average Depth) =", (max(depths), min(depths), sum(depths)/len(depths)))

100%|███████████████████████████████████████| 1536/1536 [00:28<00:00, 53.94it/s]

NCgreedy ESR
(Max Depth, Min Depth, Average Depth) = (7, 3, 4.291666666666667)


In [30]:
depths = []

for comb in tqdm(combinations):
    #creating delta
    delta = pd.DataFrame([list(comb)], columns=list(S.columns[:-1]))
    
    delta = delta.loc[0]
    
    alg = A_C_N(C="AD", N="greedy")
    
    depth, _ = alg.solve(S, delta=delta)
    
    depths.append(depth)
    
print("NCgreedy AD")    
print("(Max Depth, Min Depth, Average Depth) =", (max(depths), min(depths), sum(depths)/len(depths)))

100%|█████████████████████████████████████████████| 2/2 [00:00<00:00, 55.74it/s]

NCgreedy AD
(Max Depth, Min Depth, Average Depth) = (5, 4, 4.5)


In [31]:
depths = []

for comb in tqdm(extended_combinations):
    #creating delta
    delta = pd.DataFrame([list(comb)], columns=list(S.columns[:-1]))
    
    delta = delta.loc[0]
    
    alg = A_C_N(C="EAD", N="greedy")
    
    depth, _ = alg.solve(S, delta=delta)
    
    depths.append(depth)
    
print("NCgreedy EAD")    
print("(Max Depth, Min Depth, Average Depth) =", (max(depths), min(depths), sum(depths)/len(depths)))

100%|███████████████████████████████████████| 1536/1536 [00:25<00:00, 60.91it/s]

NCgreedy EAD
(Max Depth, Min Depth, Average Depth) = (7, 3, 4.375)


# Greedy

In [32]:
depths = []

for comb in tqdm(combinations):
    #creating delta
    delta = pd.DataFrame([list(comb)], columns=list(S.columns[:-1]))
    
    delta = delta.loc[0]
    
    alg = A_C_G(C="AR")
    
    depth, _ = alg.solve(S, delta=delta)
    
    depths.append(depth)
    
print("Greedy AR")    
print("(Max Depth, Min Depth, Average Depth) =", (max(depths), min(depths), sum(depths)/len(depths)))

100%|█████████████████████████████████████████████| 2/2 [00:00<00:00, 95.48it/s]

Greedy AR
(Max Depth, Min Depth, Average Depth) = (7, 7, 7.0)


In [33]:
depths = []

for comb in tqdm(extended_combinations):
    #creating delta
    delta = pd.DataFrame([list(comb)], columns=list(S.columns[:-1]))
    
    delta = delta.loc[0]
    
    alg = A_C_G(C="EAR")
    
    depth, _ = alg.solve(S, delta=delta)
    
    depths.append(depth)
    
print("Greedy EAR")    
print("(Max Depth, Min Depth, Average Depth) =", (max(depths), min(depths), sum(depths)/len(depths)))

100%|██████████████████████████████████████| 1536/1536 [00:08<00:00, 184.51it/s]

Greedy EAR
(Max Depth, Min Depth, Average Depth) = (7, 2, 4.625)


In [34]:
depths = []

for comb in tqdm(combinations):
    #creating delta
    delta = pd.DataFrame([list(comb)], columns=list(S.columns[:-1]))
    
    delta = delta.loc[0]
    
    alg = A_C_G(C="SR")
    
    depth, _ = alg.solve(S, delta=delta)
    
    depths.append(depth)
    
print("Greedy SR")    
print("(Max Depth, Min Depth, Average Depth) =", (max(depths), min(depths), sum(depths)/len(depths)))

100%|█████████████████████████████████████████████| 2/2 [00:00<00:00, 97.29it/s]

Greedy SR
(Max Depth, Min Depth, Average Depth) = (4, 1, 2.5)


In [35]:
depths = []

for comb in tqdm(extended_combinations):
    #creating delta
    delta = pd.DataFrame([list(comb)], columns=list(S.columns[:-1]))
    
    delta = delta.loc[0]
    
    alg = A_C_G(C="ESR")
    
    depth, _ = alg.solve(S, delta=delta)
    
    depths.append(depth)
    
print("Greedy ESR")    
print("(Max Depth, Min Depth, Average Depth) =", (max(depths), min(depths), sum(depths)/len(depths)))

100%|██████████████████████████████████████| 1536/1536 [00:12<00:00, 124.59it/s]

Greedy ESR
(Max Depth, Min Depth, Average Depth) = (7, 1, 2.65625)


In [36]:
depths = []

for comb in tqdm(combinations):
    #creating delta
    delta = pd.DataFrame([list(comb)], columns=list(S.columns[:-1]))
    
    delta = delta.loc[0]
    
    alg = A_C_G(C="AD")
    
    depth, _ = alg.solve(S, delta=delta)
    
    depths.append(depth)
    
print("Greedy AD")    
print("(Max Depth, Min Depth, Average Depth) =", (max(depths), min(depths), sum(depths)/len(depths)))

100%|████████████████████████████████████████████| 2/2 [00:00<00:00, 108.71it/s]

Greedy AD
(Max Depth, Min Depth, Average Depth) = (4, 4, 4.0)


In [37]:
depths = []

for comb in tqdm(extended_combinations):
    #creating delta
    delta = pd.DataFrame([list(comb)], columns=list(S.columns[:-1]))
    
    delta = delta.loc[0]
    
    alg = A_C_G(C="EAD")
    
    depth, _ = alg.solve(S, delta=delta)
    
    depths.append(depth)
    
print("Greedy EAD")    
print("(Max Depth, Min Depth, Average Depth) =", (max(depths), min(depths), sum(depths)/len(depths)))

100%|██████████████████████████████████████| 1536/1536 [00:10<00:00, 140.99it/s]

Greedy EAD
(Max Depth, Min Depth, Average Depth) = (7, 2, 3.2395833333333335)
